# Webscrape Project
This script will demonstrate usage of the webscape function in python

Stats will be taken from basketball-reference.com and inserted into a SQLite database. A SQL script will extract and translate the data for analysis.
Analaysis of the data will primarily focus on determining what stats are key influences on the outcome of Bucks performances (W/L)

In [1]:
#import libraries
import requests
from bs4 import BeautifulSoup

Beautiful Soup is the parser, requests is the url getter. Now the url to be scraped is defined.

In [2]:
#Url Definition
url = "https://www.basketball-reference.com/teams/MIL/2025/gamelog/#team_game_log_reg::31"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
                   (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"
}

#try the website and see the response

try:
    # Send the request
    response = requests.get(url, headers=headers)
    
    # Check for successful request
    if response.status_code != 200:
        print(f"Failed to retrieve the page: {response.status_code}")
    else:
        html_content = response.text

        # Parse HTML with BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract page title
        title = soup.title.text if soup.title else "No title found"
        print("Page Title:", title)

        # Extract and print all paragraphs
        paragraphs = soup.find_all('p')
        for idx, para in enumerate(paragraphs, 1):
            print(f"Paragraph {idx}:", para.get_text(strip=True))
            
        # Example of waiting before making another request
        time.sleep(2)  # Wait for 2 seconds
        
except Exception as e:
    print("An error occurred:", e)

Page Title: 2024-25 Milwaukee Bucks Team Game Log | Basketball-Reference.com
Paragraph 1: via Sports Logos.net
Paragraph 2: About logos
Paragraph 3: Record:47-34, 5th inNBAEastern Conference
Paragraph 4: Last Game:W 125-119 
         at  
          DET
Paragraph 5: Next Game:Sunday, Apr. 13 
         vs.  
          DET
Paragraph 6: Coach:Doc Rivers(47-34)
Paragraph 7: Executive:Jon Horst
Paragraph 8: PTS/G:115.2 (12th of 30)Opp PTS/G:112.8 (13th of 30)
Paragraph 9: SRS: 2.04 (11th of 30)Pace: 99.3 (12th of 30)
Paragraph 10: Off Rtg: 115.7 (12th of 30)Def Rtg: 113.3 (12th of 30)Net Rtg: +2.4 (11th of 30)
Paragraph 11: Expected W-L: 46-35 (11th of 30)
Paragraph 12: Arena:Fiserv ForumAttendance:714,218 (17th of 30)
Paragraph 13: Click two rows to sum games. Click on the Gtm value to see the team's season totals through that game.
Paragraph 14: Bucks Franchise Index
Paragraph 15: EverySports Reference Social Media Account
Paragraph 16: Site Last Updated:Sunday, April 13,  7:52PM
Paragraph

The headers portion of this script is like what you say when you ring someones doorbell. In this case its telling the website hey I am from a regular browser. This is to prevent the website from responding with "no we dont talk to robots"

In [3]:
soup


<!DOCTYPE html>

<html class="no-js" data-root="/home/bbr/deploy/www" data-version="klecko-" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport">
<link href="https://cdn.ssref.net/req/202504030" rel="dns-prefetch"/>
<script>
/* https://docs.osano.com/hc/en-us/articles/22469433444372-Google-Consent-Mode-v2  */
  window.dataLayer = window.dataLayer ||[];
      function gtag(){dataLayer.push(arguments);}
      gtag('consent','default',{
        'ad_storage':'denied',
        'analytics_storage':'denied',
        'ad_user_data':'denied',
        'ad_personalization':'denied',
        'personalization_storage':'denied',
        'functionality_storage':'granted',
        'security_storage':'granted',
        'wait_for_update': 500
      });
      gtag("set", "ads_data_redaction", true);
</script>
<script src="https://cmp.osano.com/16CGnCU8UtNhM14sg/12669873-8

That works for the basic text scraping. However, this web browser has a table that I believe to be operated thru Java script. I thought I would be out of luck since I don't know Java yet, however I found a button that converts the table to CSV format. I am going to use Selenium to click the button.

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize Safari WebDriver
driver = webdriver.Safari()

# Navigate to the target webpage
url = "https://www.basketball-reference.com/teams/MIL/2025/gamelog/#team_game_log_reg::31"
driver.get(url)

# Wait helper with a timeout (e.g., 10 seconds)
wait = WebDriverWait(driver, 10)

# Locate the dropdown menu element that contains "Share & Export"
share_export_menu = wait.until(
    EC.visibility_of_element_located(
        (By.XPATH, '//li[contains(@class,"hasmore") and .//span[contains(text(),"Share & Export")]]')
    )
)

# Scroll the dropdown element into view
driver.execute_script("arguments[0].scrollIntoView(true);", share_export_menu)
time.sleep(1)

# Hover over the dropdown so that its contents appear
actions = ActionChains(driver)
actions.move_to_element(share_export_menu).perform()
time.sleep(1)
#IF WANT TO USE CSV, ADD CSV CODE HERE

#IF WANT TO USE EXPORT AS, ADD EXPORT AS CODE HERE

# Use a more robust locator: locate the button using its tip attribute text.
# This button's tip contains "Export table as"
export_button = wait.until(
    EC.presence_of_element_located(
        (By.XPATH, '//button[contains(@tip, "Export table as")]')
    )
)

# Scroll the export button into view in case it's hidden
driver.execute_script("arguments[0].scrollIntoView(true);", export_button)
time.sleep(0.5)

# Click the export button using JavaScript to bypass any clickable issues
driver.execute_script("arguments[0].click();", export_button)
time.sleep(3)

# Optional: wait a few seconds before closing the browser
time.sleep(3)

# Close the browser
driver.quit()

# END ADDED CODE HERE

the portion of code below can be used to click the csv button and scrape the data from the csv pre text

In [ ]:
# Use a more robust locator: locate the button using its tip attribute text.
# This button's tip contains "Convert the table below to comma-separated values"
csv_button = wait.until(
    EC.presence_of_element_located(
        (By.XPATH, '//button[contains(@tip, "Convert the table below to comma-separated values")]')
    )
)

# Scroll the CSV button into view in case it's hidden
driver.execute_script("arguments[0].scrollIntoView(true);", csv_button)
time.sleep(0.5)

# Click the CSV button using JavaScript to bypass any clickable issues
driver.execute_script("arguments[0].click();", csv_button)

# After clicking the CSV button, wait until the CSV data appears
pre_element = wait.until(
    EC.visibility_of_element_located((By.ID, "csv_team_game_log_reg"))
)

# Extract the CSV text from the <pre> tag.
csv_data = pre_element.text
print("CSV Data extracted:")
print(csv_data)

# Optional: wait a few seconds before closing the browser
time.sleep(3)

# Close the browser
driver.quit()


the portion of code below allows to export the table as an excel workbook

In [ ]:
# Use a more robust locator: locate the button using its tip attribute text.
# This button's tip contains "Export table as"
export_button = wait.until(
    EC.presence_of_element_located(
        (By.XPATH, '//button[contains(@tip, "Export table as")]')
    )
)

# Scroll the export button into view in case it's hidden
driver.execute_script("arguments[0].scrollIntoView(true);", export_button)
time.sleep(0.5)

# Click the export button using JavaScript to bypass any clickable issues
driver.execute_script("arguments[0].click();", export_button)
time.sleep(3)

# Optional: wait a few seconds before closing the browser
time.sleep(3)

# Close the browser
driver.quit()

Now the data is loaded into the Pandas dataframe and then into the SQLite database.

In [14]:
# Now, load the CSV text into a Pandas DataFrame
csv_buffer = StringIO(csv_data)
df = pd.read_csv(csv_buffer)

# Connect to (or create) an SQLite database and insert the DataFrame into a table
db_name = "NBA_25.db"      # name of your database file
table_name = "TeamStats"            # name of the table in the database
conn = sqlite3.connect(db_name)
df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()

print(f"Data inserted into SQLite database '{db_name}' in table '{table_name}'.")

Data inserted into SQLite database 'NBA_25.db' in table 'TeamStats'.


Adding this line to test git commit function in terminal